<a href="https://colab.research.google.com/github/le-Mon94/HateOffensiveSpeech-Detector-FromAudio/blob/main/Gradio_app_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/le-Mon94/HateOffensiveSpeech-Detector-FromAudio.git

Cloning into 'HateOffensiveSpeech-Detector-FromAudio'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 149 (delta 17), reused 16 (delta 7), pack-reused 116
Receiving objects: 100% (149/149), 7.57 MiB | 13.58 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [ ]:
!pip install pydub
!pip install gradio
!pip install git+https://github.com/openai/whisper.git
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
 

In [ ]:
import joblib
import gradio as gr
import pandas as pd
from pydub import AudioSegment
import os
import math
import whisper
import numpy as np

import statistics
from statistics import mode

In [ ]:
WhisperModel = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:25<00:00, 59.6MiB/s]


In [ ]:
def split_text(text):
    return text.split()

PredictModel = joblib.load('/content/HateOffensiveSpeech-Detector-FromAudio/trained_detection_model.joblib')

In [ ]:
def split_sentence(sentence):
    sentences = sentence.split('.')

    sentences = [s.strip() for s in sentences if s.strip()]

    return sentences

In [ ]:
def PredictFunction(audio_file):

  label_mapping = label_mapping = {0: 'Hate', 1: 'Offensive', 2: 'No Hate / Offensive'}

  text_result = WhisperModel.transcribe(audio_file)

  result_list = split_sentence(text_result["text"])

  print(text_result["text"])

  pred = PredictModel.predict(result_list)

  pred_labels = [label_mapping[label] for label in pred]

  result_df = pd.DataFrame({'text': result_list, 'predicted_label': pred_labels})

  result_mode = mode(pred_labels)

  return result_df, "This audio is contain " + result_mode + " speeches"

In [ ]:
demo = gr.Blocks()

mic_transcribe = gr.Interface(
    fn=PredictFunction,
    inputs=gr.Audio(source="microphone", type="filepath", label="Record an audio with your mic"),
    outputs=[gr.Dataframe(label="Prediction DataFrame"), gr.outputs.Textbox(label="Result")],
    title="Hate / Offensive Speech Detector From Audio",
    description="Detect Hate and Offensive speeches using Whisper AI to transcribe the audio and pretrained model from skleanr and LinearSVC to detect it"
)

file_transcribe = gr.Interface(
    fn=PredictFunction,
    inputs=gr.Audio(source="upload", type="filepath", label="Browse and upload your audio"),
    outputs=[gr.Dataframe(label="Prediction DataFrame"), gr.outputs.Textbox(label="Result")],
    title="Hate / Offensive Speech Detector From Audio",
    description="Detect Hate and Offensive speeches using Whisper AI to transcribe the audio and pretrained model from sklearn and LinearSVC to detect it"
)

<ipython-input-11-90c393b0172a>:6: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=[gr.Dataframe(label="Prediction DataFrame"), gr.outputs.Textbox(label="Result")],
<ipython-input-11-90c393b0172a>:14: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=[gr.Dataframe(label="Prediction DataFrame"), gr.outputs.Textbox(label="Result")],


In [ ]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe, file_transcribe],
        ["Transcribe Microphone", "Transcribe Audio File"],
    )

demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:954: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
